In [1]:
# imports
from datamodel import OrderDepth, UserId, TradingState, Order, Observation, ConversionObservation
from typing import List, Dict, Tuple, Any
import string
import json
import jsonpickle
import numpy as np
import math
from math import log, sqrt, exp
from dataclasses import dataclass
from statistics import NormalDist
import random
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
file_path = "/Users/vasudev/Desktop/Projects/imc_prosperity/prosperity-imc25/rounds/round5/eda/logs/61789b7f-b8f8-4453-a5a9-823f5133b2c5.log"

In [3]:
@dataclass
class Trade:
    timestamp: int
    buyer: str
    seller: str
    symbol: str
    currency: str
    price: float
    quantity: int

def parse_logs(file_path: str) -> Tuple[str, pd.DataFrame, List[Any]]:
    # Read entire log file
    with open(file_path, 'r') as f:
        content = f.read()

    # --- Parse Sandbox Logs ---
    sandbox_start = content.find('Sandbox logs:')
    activities_start = content.find('Activities log:')
    sandbox_section = content[sandbox_start + len('Sandbox logs:'):activities_start].strip()

    # --- Parse Activities Log ---
    trades_start = content.find('Trade History:')
    activities_section = content[activities_start + len('Activities log:'):trades_start].strip()

    # Read CSV section into a DataFrame
    activities_df = pd.read_csv(StringIO(activities_section), sep=';')

    # --- Parse Trades ---
    trades_section = content[trades_start + len('Trade History:'):].strip()
    trades_data = json.loads(trades_section)

    # Convert to Trade objects
    trades: List[Any] = [Trade(**t) for t in trades_data]

    return sandbox_section, activities_df, trades

sandbox_section, activities_df, trades = parse_logs(file_path)

In [4]:
trades_df = pd.DataFrame([t.__dict__ for t in trades])
our_trades = [trade for trade in trades if trade.buyer == "SUBMISSION" or trade.seller == "SUBMISSION"]
our_trades_df = pd.DataFrame([trade.__dict__ for trade in our_trades])
print(len(our_trades_df))

5


In [5]:
# Sort the trades chronologically
our_trades_sorted = sorted(our_trades, key=lambda x: x.timestamp)

# Extract all unique timestamps and products from the activities dataframe
timestamps = sorted(activities_df['timestamp'].unique())
symbols = activities_df['product'].unique()

# Keep track of running position
running_pos = defaultdict(int)
records = []

# Pointer for iterating through our_trades_sorted
t_idx = 0

for t in timestamps:
    # Apply all trades that occurred at this timestamp
    while t_idx < len(our_trades_sorted) and our_trades_sorted[t_idx].timestamp == t:
        trade = our_trades_sorted[t_idx]
        if trade.buyer == "SUBMISSION":
            running_pos[trade.symbol] += trade.quantity
        elif trade.seller == "SUBMISSION":
            running_pos[trade.symbol] -= trade.quantity
        t_idx += 1

    # Record the position for all products at this timestamp
    for symbol in symbols:
        records.append({'timestamp': t, 'product': symbol, 'position': running_pos[symbol]})

# Create DataFrame of positions and merge
position_df = pd.DataFrame(records)
activities_df = activities_df.merge(position_df, on=['timestamp', 'product'], how='left')

# PRODUCT DICT

In [6]:
products = [
    "RAINFOREST_RESIN",
    "KELP", "SQUID_INK",
    "CROISSANTS", "JAMS",
    "DJEMBES", "PICNIC_BASKET1",
    "PICNIC_BASKET2",
    "VOLCANIC_ROCK",
    "VOLCANIC_ROCK_VOUCHER_9500", "VOLCANIC_ROCK_VOUCHER_9750",
    "VOLCANIC_ROCK_VOUCHER_10000", "VOLCANIC_ROCK_VOUCHER_10250",
    "VOLCANIC_ROCK_VOUCHER_10500", "MAGNIFICENT_MACARONS"
]

product_dict = {
    product: {
        "order_book": activities_df[activities_df['product'] == product],
        "trades": trades_df[trades_df['symbol'] == product]
    }
    for product in products
}
# check length of product_dict
print("Length of product_dict:", len(product_dict))

Length of product_dict: 15


# POSITIONS

In [8]:
product_dict["RAINFOREST_RESIN"]["order_book"].head(10)

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss,position
12,4,0,RAINFOREST_RESIN,9992.0,35.0,NaN,NaN,NaN,NaN,10008,35,NaN,NaN,NaN,NaN,10000.0,0.0,0
16,4,100,RAINFOREST_RESIN,9992.0,30.0,NaN,NaN,NaN,NaN,10008,30,NaN,NaN,NaN,NaN,10000.0,0.0,0
43,4,200,RAINFOREST_RESIN,9992.0,27.0,NaN,NaN,NaN,NaN,10008,27,NaN,NaN,NaN,NaN,10000.0,0.0,0
49,4,300,RAINFOREST_RESIN,9992.0,31.0,NaN,NaN,NaN,NaN,10008,31,NaN,NaN,NaN,NaN,10000.0,0.0,0
64,4,400,RAINFOREST_RESIN,9992.0,27.0,NaN,NaN,NaN,NaN,10008,27,NaN,NaN,NaN,NaN,10000.0,0.0,0
87,4,500,RAINFOREST_RESIN,9992.0,26.0,NaN,NaN,NaN,NaN,10008,26,NaN,NaN,NaN,NaN,10000.0,0.0,0
95,4,600,RAINFOREST_RESIN,9992.0,27.0,NaN,NaN,NaN,NaN,10008,27,NaN,NaN,NaN,NaN,10000.0,0.0,0
115,4,700,RAINFOREST_RESIN,9992.0,32.0,NaN,NaN,NaN,NaN,10008,32,NaN,NaN,NaN,NaN,10000.0,0.0,0
133,4,800,RAINFOREST_RESIN,9992.0,30.0,NaN,NaN,NaN,NaN,9999,1,10008.0,30.0,NaN,NaN,9995.5,0.0,-30
136,4,900,RAINFOREST_RESIN,9997.0,6.0,9992.0,34.0,NaN,NaN,10008,34,NaN,NaN,NaN,NaN,10002.5,-240.0,-30


In [7]:
plt.figure(figsize=(15, 8))

for product, df in product_dict.items():
    # Plot position
    plt.plot(df['timestamp'], df['position'], label=product)
    plt.xlabel('Timestamp')
    plt.ylabel('Position')
    plt.title('Position Over Time')
    plt.legend()
    plt.grid()


KeyError: 'timestamp'

<Figure size 1500x800 with 0 Axes>

# SQUID INK POSITIONS